In [17]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [18]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [19]:
question_class = 'q1'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of content', len(content)
f.close()
print content

length of content 168
['after taking appointment then also i have to wait 1 to 1 and half hour . even there was no rush , work should be done immediately .', 'service center should provide comfortable time for servicing from there end . or they should arrange service like we can do booking through messages . also toyota has given only one number for all services .', 'after arranging the appointment they should quickly service the car and also delivery of car should fast .', 'they pick up the phone and then disconnect it . they tell their schedule is busy , they say it will take some time we will call you but they never call back . and when we call the service station then immediately they ask us to send the car . they should tell us the schedule properly .', 'toyota have an app , it \\? s a mobile app which is not applicable for all the customers . first of all , this app should be for every customer . this app should be available for all the users . also this app should be free for al

In [20]:
# split comment with multi-sentence into multi-comments
content_new = []
for comment in content:
    sents = sent_tokenize(comment)
    if len(sents) > 1:
        for i in range(len(sents)):
            content_new.append(sents[i])
    else:
        content_new.append(comment)
print 'length of new content', len(content_new)
# print content_new
    

length of new content 395


In [28]:
stemmer = PorterStemmer()
print stemmer.stem("ungency")

ungenc


In [22]:

def rule_q1(sen, ne):
    clean_ne = list(set(ne))
    # remove_words = ['customer', 'customers', 'waiting', 'facility', 'facilities', 'dealership', 'toyota', 'improvement']
    remove_words = ["appoint", "improv", "custom", "servic", "peopl","person","facil","avail","good",\
                    "center","centr","car", "dealership", "vehicl", "toyota", "problem","work", "much",\
                   "thing", "possibl","need"]   #stemmed
    clean_ne = [word for word in clean_ne if word not in remove_words]
    
    save_words = ["without", "call","wait", "pick", "pickup", "drop","remind", "inform", "respons", "book","fix",\
                 "receiv", "same", "sm", "immedi", "urgent","urgenc","deliv","deliveri","explain", "detail"]  #stemmed
    clean_ne = clean_ne + [stemmer.stem(word) for word in sen.split() if stemmer.stem(word) in save_words]
    
    # rules to merge keywords:
    if 'pickup' in clean_ne:
        clean_ne[clean_ne.index('pickup')] = 'pick'
    if 'phone' in clean_ne:
        clean_ne[clean_ne.index('phone')] = 'call'
    if 'telephon' in clean_ne:
        clean_ne[clean_ne.index('telephon')] = 'call'
    if 'messag' in clean_ne:
        clean_ne[clean_ne.index('messag')] = 'sm'
    if 'km' in clean_ne:
        clean_ne[clean_ne.index('km')] = 'locat'
    if 'area' in clean_ne:
        clean_ne[clean_ne.index('area')] = 'locat'
    if 'deliveri' in clean_ne:
        clean_ne[clean_ne.index('deliveri')] = 'deliv'   
    if 'urgent' in clean_ne:
        clean_ne[clean_ne.index('urgent')] = 'emerg'
    if 'urgenc' in clean_ne:
        clean_ne[clean_ne.index('urgenc')] = 'emerg'
    if 'detail' in clean_ne:
        clean_ne[clean_ne.index('detail')] = 'inform'
    if 'explain' in clean_ne:
        clean_ne[clean_ne.index('explain')] = 'inform'
    if 'should tell' in sen:
        clean_ne.append('inform')
    # add keyword "appointment"
    if 'book' in clean_ne:
        clean_ne[clean_ne.index('book')] = 'appointment'
    if 'day' in clean_ne:  # split 'day' to 'appointment' and 'wait'
        if 'appointment' in sen:
            clean_ne[clean_ne.index('day')] = 'appointment'
        elif 'days' in sen:
            clean_ne[clean_ne.index('day')] = 'wait'
        else:
            clean_ne.remove('day')
    if 'week' in clean_ne and 'appointment' in sen:
        clean_ne[clean_ne.index('week')] = 'appointment'
    if 'month' in clean_ne and 'appointment' in sen:
        clean_ne[clean_ne.index('month')] = 'appointment'
    # add keyword "without_appointment"
    if 'without appointment' in sen or 'without any appointment' in sen:
        clean_ne.append('without_appointment')
    # add keyword "on_time"
    if 'on time' in sen and 'time' in clean_ne:
        clean_ne[clean_ne.index('time')] = 'on_time'
    if 'that time' in sen and ('servic' in sen or 'work' in sen or 'attend' in sen) and 'time' in clean_ne:
        clean_ne[clean_ne.index('time')] = 'on_time'
    if 'fulfill' in sen or 'stick' in sen and 'time' in clean_ne:
        clean_ne[clean_ne.index('time')] = 'on_time'
    # split "immediately" into "wiat" and "appointment" cluster
    if 'immedi' in clean_ne:
        if 'servic' in sen or 'work' in sen or 'attend' in sen:
            clean_ne[clean_ne.index('immedi')] = 'wait'
        else:
            clean_ne[clean_ne.index('immedi')] = 'appointment'
    # split 'time' to other clusters
    if 'too much time' in sen:
        clean_ne[clean_ne.index('time')] = 'wait'
    if 'long time' in sen:
        if 'appointment' in sen:
            clean_ne[clean_ne.index('time')] = 'appointment'
        else:
            clean_ne[clean_ne.index('time')] = 'wait'
    if 'time' in clean_ne:
        if 'appointment' in sen:
            clean_ne[clean_ne.index('time')] = 'appointment'
        else:
            clean_ne.remove('time')   # remove other 'times'
        
    clean_ne = list(set(clean_ne))
    return clean_ne

def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def process_corpus(x1, pos_tags, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    for x in x1:
        if 'no improvement' in x:
            doc_noimprove.append(x)
        else:
                nn_list = []
                sen = x
                pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
                for token in pos_new:
                    if token[1] in pos_tags and not token[0] in general_stop:
                        nn_list.append(token[0])
                #stemming
                for counter,word in enumerate(nn_list):
                    nn_list[counter] = stemmer.stem(word)
                # apply rule
                nn_list = rule_q1(sen, nn_list)
                    
                if nn_list != []:
                    nn_extracted.append(nn_list)
                    doc_nn.append(sen)
                else:
                    doc_other.append(sen)
    return doc_noimprove, [doc_nn, nn_extracted], doc_other
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]
for punctuation in punctuation_list:
    stop_words.append(punctuation)

pos_tags = ['NN', 'NNS','JJ','JJR','JJS']

doc1, doc2, doc3 = process_corpus(content_new, pos_tags, stop_words)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'signle sentence with nn', len(doc_nn)
print 'no comments', len(doc1)
print 'single sentence without nn', len(doc3)
print nn_extracted

signle sentence with nn 326
no comments 9
single sentence without nn 60
[['hour', 'wait'], ['rush', 'wait'], [u'comfort', 'end'], ['appointment', 'sm'], ['number'], [u'deliveri'], ['call', 'pick'], [u'schedul', u'busi', 'call'], ['station', 'call', 'wait'], [u'schedul', 'inform'], ['app'], [u'applic', 'app', u'mobil'], ['app'], ['app', u'user'], ['app', u'user', 'free'], ['app', 'fix'], ['expert', u'advic'], [u'inform', u'proactiv'], ['driven', u'kilomet'], ['app', u'mobil', u'fix'], ['care'], ['appointment', 'last'], ['owner', 'consult'], ['wait', 'staff', 'take', 'on_time', u'unnecessari'], ['market'], ['renold', 'manner', u'respons', 'quick', u'compani', 'treat'], ['jd', u'locat'], ['showroom', 'call', 'number', 'pick'], [u'citi', 'showroom', 'number'], [u'difficulti', 'number'], ['frame', 'on_time'], [u'citi', 'on_time'], [u'procedur'], [u'mani', 'long', 'wait'], ['appointment'], [u'attent'], [u'fix'], ['way'], ['proper', 'call', 'take', 'on_time'], ['wait'], ['home', 'locat', 'jai

In [23]:
from collections import Counter
def df_count(x1):
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df
df = df_count(nn_extracted)
print df

Counter({'appointment': 62, 'call': 49, 'wait': 44, 'on_time': 24, 'pick': 22, 'drop': 17, 'number': 14, u'locat': 13, 'staff': 12, 'hour': 12, 'fix': 11, 'inform': 11, u'emerg': 10, u'receiv': 9, u'book': 8, 'free': 7, 'app': 7, 'same': 7, u'mobil': 6, u'respons': 5, 'pune': 5, 'long': 5, u'conveni': 5, u'deliv': 5, u'onlin': 5, 'sm': 5, 'month': 4, u'busi': 4, u'earli': 4, 'proper': 4, 'space': 4, 'pickup': 4, 'job': 4, 'care': 4, 'half': 3, 'quick': 3, u'kilomet': 3, 'last': 3, 'place': 3, u'schedul': 3, 'advisor': 3, 'fast': 3, 'next': 3, 'station': 3, u'morn': 3, u'friend': 3, u'part': 3, u'compani': 3, u'provid': 3, 'give': 3, 'without': 3, 'take': 3, u'user': 3, 'without_appointment': 3, 'request': 3, u'easi': 2, 'local': 2, 'period': 2, 'fullest': 2, u'banara': 2, 'remind': 2, 'crowd': 2, 'home': 2, u'awar': 2, u'abl': 2, 'showroom': 2, u'situat': 2, u'differ': 2, u'satisfi': 2, 'spare': 2, 'card': 2, u'outsid': 2, u'mani': 2, u'confus': 2, 'first': 2, 'mr': 2, 'system': 2, 'ma

In [24]:
import heapq
import os
def filter_rule_q1(doc, xth, test_corpus, original_corpus = doc_nn):
    apply_rule = False
    # print original_corpus[xth]
    # rules (priority from high to low)
    if 'app' in doc:
        test_corpus[xth] = ['app']
        print "rule_app applied"
        apply_rule = True
    elif 'without_appointment' in doc:
        test_corpus[xth] = ['without_appointment']
        print "rule_without_appointment applied"
        apply_rule = True
    elif 'sm' in doc:
        test_corpus[xth] = ['sm']
        print "rule_sm applied"
        apply_rule = True
    elif 'onlin' in doc:
        test_corpus[xth] = ['onlin']
        print "rule_onlin applied"
        apply_rule = True
    elif 'emerg' in doc:
        test_corpus[xth] = ['emerg']
        print "rule_emerg applied"
        apply_rule = True
    elif 'deliv' in doc:
        test_corpus[xth] = ['deliv']
        print "rule_deliv applied"
        apply_rule = True
    elif 'inform' in doc:
        test_corpus[xth] = ['inform']
        print "rule_inform applied"
        apply_rule = True
    elif 'fix' in doc:
        test_corpus[xth] = ['fix']
        print "rule_fix applied"
        apply_rule = True
    elif 'pick' in doc and 'drop' in doc:
        test_corpus[xth] = ['pick']
        print "rule_pick applied"
        apply_rule = True
    elif 'wait' in doc and 'month' not in original_corpus[xth]:
        test_corpus[xth] = ['wait']
        print "rule_wait applied"
        apply_rule = True
    elif 'appointment' in doc or 'day' in doc or 'week' in doc and 'call' in doc: # from call to appointment
        test_corpus[xth] = ['appointment']
        print "rule_appointment applied"
        apply_rule = True
        
    return apply_rule
    
def filter_ne(test_corpus, df):  # assuming each review contain one aspect
    for xth, doc in enumerate(test_corpus):
        if len(doc)>1:
            apply_rule = filter_rule_q1(doc, xth, test_corpus)
            if not apply_rule:
                df_words = [df[word] for word in doc]
                idx =  heapq.nlargest(1, xrange(len(df_words)), key=df_words.__getitem__)
                test_corpus[xth] = [doc[ith] for ith in idx]
    return test_corpus
nn_clean = filter_ne(nn_extracted, df)
df = df_count(nn_clean)
dict_map = dict(df.most_common())
print df

def write_file(corpus, idx_list, word):
    if not os.path.isdir("cluster/%s" %word):
            os.mkdir("cluster/%s" %word)
    f1 = open('cluster/%s/%s_comment.txt' % (word, word), 'w+')
    for idx in idx_list:
        f1.write('%s\n' %corpus[idx])
    f1.close()
    
    
def main_category(df_list, nn_clean, corpus):
    if (not os.path.isdir("cluster")):
        os.mkdir("cluster")
    name_list = {}
    clustered_index = []
    major_list = [word for word in df_list if df_list[word]>2]
    print "majot list:\n", major_list
    for word in major_list:
        idx_set = []
        for idx, doc in enumerate(nn_clean):
            if word in doc:
                idx_set.append(idx)
        write_file(doc_nn, idx_set, word)
        name_list[word] = idx_set
        clustered_index = clustered_index + idx_set
    clustered_index = set(clustered_index)
    print "clustered index:", list(clustered_index)
    unclustered_index = list(set(range(len(nn_clean))) - clustered_index)
    print "others index:", unclustered_index
    name_list['others'] = unclustered_index
    write_file(doc_nn, unclustered_index, 'others')
    return name_list

name_list = main_category(dict_map, nn_clean,doc_nn)
print name_list['call']

rule_wait applied
rule_wait applied
rule_sm applied
rule_wait applied
rule_inform applied
rule_app applied
rule_app applied
rule_app applied
rule_app applied
rule_inform applied
rule_app applied
rule_appointment applied
rule_wait applied
rule_wait applied
rule_appointment applied
rule_wait applied
rule_pick applied
rule_wait applied
rule_wait applied
rule_wait applied
rule_without_appointment applied
rule_inform applied
rule_deliv applied
rule_appointment applied
rule_wait applied
rule_appointment applied
rule_appointment applied
rule_wait applied
rule_appointment applied
rule_inform applied
rule_appointment applied
rule_inform applied
rule_wait applied
rule_wait applied
rule_wait applied
rule_appointment applied
rule_sm applied
rule_appointment applied
rule_wait applied
rule_pick applied
rule_appointment applied
rule_wait applied
rule_pick applied
rule_emerg applied
rule_fix applied
rule_fix applied
rule_onlin applied
rule_appointment applied
rule_wait applied
rule_appointment applied